## Habitat suitability under climate change: Step 1
Demo going through parts of Step 1 of the latest coding assignment

#### Prep workspace

In [1]:
### load packages

### reproducible file paths
import os
from glob import glob
import pathlib

### gbif packages
import pygbif.occurrences as occ
import pygbif.species as species
from getpass import getpass

### unzipping and handling gbif data
import zipfile
import time

### deal with spatial data
import geopandas as gpd
import xrspatial

### deal with other types of data
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrm

### invalid geometries
from shapely.geometry import MultiPolygon, Polygon

### visualizing
import holoviews as hv
import hvplot.pandas
import hvplot.xarray

c:\Users\kjsie\anaconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#### Prep file paths

In [2]:
#### make reproducible file paths
data_dir = os.path.join(

    ### home directory
    pathlib.Path.home(),

    ### eda directory
    'earth-analytics',
    'data',

    ### project dir
    'hab_suit'
)

### make the dir
os.makedirs(data_dir, exist_ok=True)

Study species: Lupinus argenteus (silvery lupine)

In [3]:
### set gbif dir
gbif_dir = os.path.join(data_dir, 'gbif_lupine')

In [4]:
### access gbif
reset_credentials = False

### enter gbif username, password, and email
credentials = dict(
    GBIF_USER=(input, 'GBIF username:'),
    GBIF_PWD=(getpass, 'GBIF password'),
    GBIF_EMAIL=(input, 'GBIF email'),
)
for env_variable, (prompt_func, prompt_text) in credentials.items():

    ### delete credential from the environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    
    ### ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

#### Set up species info for GBIF

In [5]:
### species names
species_name = 'Lupinus argenteus'

### species info for gbif
species_info = species.name_lookup(species_name, 
                                   rank = 'SPECIES')

### grab the first result
first_result = species_info['results'][0]

### get species key
species_key = first_result['nubKey']

### check on that
first_result['species'], species_key

('Lupinus argenteus', 2964374)

In [6]:
### assign species code
species_key = 2964374

#### Download species pccurrence data

In [ ]:
### set a file pattern
gbif_pattern = os.path.join(gbif_dir,
                            '*.csv')

### download it once
if not glob(gbif_pattern):

    ### submit my query to GBIF
    gbif_query = occ.download([
        f"speciesKey = {species_key}",
        "hasCoordinate = True",
    ])

    ### only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]
        download_key = os.environ['GBIF_DOWNLOAD_KEY']

        ### wait for download to build
        wait = occ.download_meta(download_key)['status']
        while not wait == 'SUCCEEDED':
            wait = occ.download_meta(download_key)['status']
            time.sleep(5)
    
    ### download the data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'],
        path = data_dir
    )

    ### unzip it
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path = gbif_dir)


### find csv file path
gbif_path = glob(gbif_pattern)[0]


INFO:Your download key is 0000570-250225214225278
INFO:Download file size: 849953 bytes
INFO:On disk at C:\Users\kjsie\earth-analytics\data\hab_suit/0000562-250225214225278.zip


#### Check out the GBIF data

In [48]:
#### open gbif data
gbif_df = pd.read_csv(
    gbif_path,
    delimiter = '\t'
)

### take a look
gbif_df.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,997427416,95c938a8-f762-11e1-a439-00145eb45e9a,91b2ca69-7e6a-416c-a7d7-e417f2b0e710,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,Mark Madsen,NaN,CC_BY_4_0,NaN,Mark Madsen,NaN,NaN,2025-02-13T16:11:37.814Z,StillImage,NaN
1,997427274,95c938a8-f762-11e1-a439-00145eb45e9a,b5a9af5a-1de7-4349-83dd-ae6f7f40d1d8,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,Mark Madsen,NaN,CC_BY_4_0,NaN,Mark Madsen,NaN,NaN,2025-02-13T16:11:44.863Z,StillImage,GEODETIC_DATUM_ASSUMED_WGS84
2,894988816,5678b0b3-450b-4513-82e1-2b32c3c50b54,8783,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,NaN,CC0_1_0,University of Lethbridge Herbarium,"J. Nagy, W. Blais",NaN,NaN,2025-02-06T17:56:06.190Z,StillImage,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
3,894988807,5678b0b3-450b-4513-82e1-2b32c3c50b54,8803,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,NaN,CC0_1_0,University of Lethbridge Herbarium,Darwyn Coxson,NaN,NaN,2025-02-06T17:56:06.228Z,StillImage,GEODETIC_DATUM_ASSUMED_WGS84;TAXON_MATCH_TAXON...
4,894988792,5678b0b3-450b-4513-82e1-2b32c3c50b54,10790,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,NaN,CC0_1_0,University of Lethbridge Herbarium,"Warrington, Nagy",NaN,NaN,2025-02-06T17:56:05.638Z,StillImage,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...


In [49]:
### see what the columns are
gbif_df.columns

Index(['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species', 'infraspecificEpithet',
       'taxonRank', 'scientificName', 'verbatimScientificName',
       'verbatimScientificNameAuthorship', 'countryCode', 'locality',
       'stateProvince', 'occurrenceStatus', 'individualCount',
       'publishingOrgKey', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation',
       'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day',
       'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord',
       'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber',
       'identifiedBy', 'dateIdentified', 'license', 'rightsHolder',
       'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted',
       'mediaType', 'issue'],
      dtype='object')

In [50]:
### make it spatial
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df,
        geometry=gpd.points_from_xy(
            gbif_df.decimalLongitude,
            gbif_df.decimalLatitude
        ),
        crs = 'EPSG:4326'
    )
)

gbif_gdf

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,geometry
0,997427416,95c938a8-f762-11e1-a439-00145eb45e9a,91b2ca69-7e6a-416c-a7d7-e417f2b0e710,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC_BY_4_0,NaN,Mark Madsen,NaN,NaN,2025-02-13T16:11:37.814Z,StillImage,NaN,POINT (-111.1946 38.096)
1,997427274,95c938a8-f762-11e1-a439-00145eb45e9a,b5a9af5a-1de7-4349-83dd-ae6f7f40d1d8,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC_BY_4_0,NaN,Mark Madsen,NaN,NaN,2025-02-13T16:11:44.863Z,StillImage,GEODETIC_DATUM_ASSUMED_WGS84,POINT (-113.8543 37.4244)
2,894988816,5678b0b3-450b-4513-82e1-2b32c3c50b54,8783,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC0_1_0,University of Lethbridge Herbarium,"J. Nagy, W. Blais",NaN,NaN,2025-02-06T17:56:06.190Z,StillImage,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...,POINT (-113.83333 49.08333)
3,894988807,5678b0b3-450b-4513-82e1-2b32c3c50b54,8803,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC0_1_0,University of Lethbridge Herbarium,Darwyn Coxson,NaN,NaN,2025-02-06T17:56:06.228Z,StillImage,GEODETIC_DATUM_ASSUMED_WGS84;TAXON_MATCH_TAXON...,POINT (-113.8 49.55)
4,894988792,5678b0b3-450b-4513-82e1-2b32c3c50b54,10790,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC0_1_0,University of Lethbridge Herbarium,"Warrington, Nagy",NaN,NaN,2025-02-06T17:56:05.638Z,StillImage,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...,POINT (-113.11667 49.78333)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8379,1039012307,963f12d0-f762-11e1-a439-00145eb45e9a,urn:uuid:e01c7cf2-4cfe-44f8-9f34-531e48453ef3,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC0_1_0,Yale Peabody Museum,Frank Tweedy,Isotype,NaN,2025-02-24T13:11:10.534Z,StillImage,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...,POINT (-110.37968 43.73772)
8380,1038955358,963f12d0-f762-11e1-a439-00145eb45e9a,urn:uuid:593834d9-1833-481d-a0cd-02c99235402f,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC0_1_0,Yale Peabody Museum,Frank Tweedy,Isotype,NaN,2025-02-25T13:11:52.632Z,StillImage,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...,POINT (-106.98227 41.15554)
8381,1038955358,963f12d0-f762-11e1-a439-00145eb45e9a,urn:uuid:593834d9-1833-481d-a0cd-02c99235402f,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC0_1_0,Yale Peabody Museum,Frank Tweedy,Isotype,NaN,2025-02-24T13:10:51.780Z,StillImage,OCCURRENCE_STATUS_INFERRED_FROM_INDIVIDUAL_COU...,POINT (-106.98227 41.15554)
8382,1030479279,c01696db-a5fe-4d3a-8b26-90760604774b,f687ac8f-3039-11e3-963d-8844b9d50285,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Lupinus,Lupinus argenteus,...,NaN,CC_BY_NC_4_0,NaN,Arnold Tiehm,NaN,NaN,2025-02-21T21:10:32.405Z,StillImage,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...,POINT (-118.23842 41.98369)


In [51]:
### plot where it's found
gbif_gdf.hvplot(
    geo=True, tiles='EsriImagery',
    title = 'Silvery lupine occurrences in GBIF',
    fill_color = None, line_color = 'purple', 
    frame_width = 600
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

## Select study sites

In [12]:
### site directory
site_dir = os.path.join(data_dir, 'sites_lupine')
os.makedirs(site_dir, exist_ok=True)

https://www.usgs.gov/programs/gap-analysis-project/science/pad-us-data-download

In [22]:
## open pa path
pa_path = os.path.join(site_dir, 'PADUS4_0_StateCA.gdb')

### open polygon
pa_shp = gpd.read_file(pa_path)

c:\Users\kjsie\anaconda3\envs\earth-analytics-python\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'PADUS4_0_StateCA.gdb': 'PADUS4_0Fee_State_CA' (default), 'PADUS4_0Designation_State_CA', 'PADUS4_0Easement_State_CA', 'PADUS4_0Proclamation_State_CA', 'PADUS4_0Comb_DOD_Trib_NGP_Fee_Desig_Ease_State_CA', 'PADUS4_0Marine', 'Public_Access', 'Agency_Name', 'Agency_Type', 'Category', 'Designation_Type', 'GAP_Status', 'IUCN_Category', 'State_Name'. Specify layer parameter to avoid this warning.
  result = read_func(


In [23]:
### check crs
print(pa_shp.crs)

PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102039"]]


In [24]:
### convert crs
pa_shp = pa_shp.to_crs(epsg = 4326)

##### deal with invalid and missing geometries

In [25]:
### fix invalid geoms
pa_shp['geometry'] = pa_shp['geometry'].apply(lambda geom: geom.make_valid() if not isinstance(geom,
                                                                                              MultiPolygon) and not geom.is_valid else geom)

In [26]:
### drop remaining invalid geometries
pa_shp = pa_shp[pa_shp.geometry.is_valid]

In [27]:
### drop rows with missing geometries
pa_shp = pa_shp.dropna(subset=['geometry'])

In [30]:
### plot a subset of the sites
subset_pa = pa_shp.head(50)
subset_pa.hvplot(
    geo=True, tiles='EsriImagery',
    title = 'Protected areas in California',
    fill_color = None, line_color = 'orange', 
    frame_width = 600
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [34]:
### check out the columns
pa_shp.columns

Index(['Own_Name', 'Mang_Name', 'Unit_Nm', 'Loc_Nm', 'geometry'], dtype='object')

In [ ]:
### simplify columns
pa_shp = pa_shp[['Own_Name', 'Mang_Name',
                 'Unit_Nm', 'Loc_Nm',
                 'geometry']]

##### Option 1: select study sites by GBIF occurrences

In [35]:
### intersect lupine occurrence with california PAs
lupine_ca = gpd.overlay(gbif_gdf, pa_shp, how = 'intersection')

In [37]:
### how many occurrences per site?
value_counts = lupine_ca['Loc_Nm'].value_counts()
value_counts

Loc_Nm
Los Angeles Department of Water and Power    7
Ahjumawi Lava Springs State Park             5
Lake McClure (Exchequer Reservoir)           5
Lower Klamath National Wildlife Refuge       4
LOWER KLAMATH NATIONAL WILDLIFE REFUGE       4
Santa Catalina Island                        2
Conway Summit                                2
Lundy Canyon Campground                      1
LABE                                         1
San Bruno Mountain State and County Park     1
Hallelujah Junction Wildlife Area            1
Sierra Valley Preserve                       1
Folsom Lake                                  1
Folsom Lake State Recreation Area            1
American River Parkway                       1
Butte Valley Wildlife Area                   1
Name: count, dtype: int64

##### Option 2: select study sites based on research into where the species is found

Here, I will focus on Inyo National Forest and Carrizo Plain National Monument, two places where I have observed this species.

For Inyo, I know that I just want the rows where Loc_Nm is "Inyo National Forest."

In [38]:
### subset to inyo
inyo_gdf = pa_shp[pa_shp['Loc_Nm'] == 'Inyo National Forest']

### drop excess columns
inyo_gdf = inyo_gdf[['Loc_Nm', 'geometry']]

inyo_gdf

,Loc_Nm,geometry
88,Inyo National Forest,"MULTIPOLYGON (((-118.55012 37.98427, -118.5469..."
104,Inyo National Forest,"MULTIPOLYGON (((-118.06059 36.60385, -118.0600..."
133,Inyo National Forest,"MULTIPOLYGON (((-119.02249 37.58457, -119.0221..."


For the Carrizo Plain, there are a couple different Loc_Nm's I might want:

In [39]:
### figure out which rows are Carrizo
carrizo_rows = pa_shp[pa_shp['Loc_Nm'].str.contains('Carrizo',
                                                    case = False,
                                                    na = False)]

### simplify cols
carrizo_rows = carrizo_rows[['Loc_Nm', 'geometry']]

### simplify loc name
carrizo_gdf = carrizo_rows.copy()
carrizo_gdf['Loc_Nm'] = 'Carrizo Plain'

carrizo_gdf

,Loc_Nm,geometry
2295,Carrizo Plain,"MULTIPOLYGON (((-119.8591 35.1903, -119.85908 ..."
2313,Carrizo Plain,"MULTIPOLYGON (((-119.85911 35.1903, -119.8591 ..."
2314,Carrizo Plain,"MULTIPOLYGON (((-119.91665 35.24888, -119.9166..."
16740,Carrizo Plain,"MULTIPOLYGON (((-119.60647 34.98708, -119.6064..."
16741,Carrizo Plain,"MULTIPOLYGON (((-119.92409 35.26665, -119.9241..."
16742,Carrizo Plain,"MULTIPOLYGON (((-119.98678 35.32143, -119.9867..."
17346,Carrizo Plain,"MULTIPOLYGON (((-119.85911 35.1903, -119.8591 ..."
17347,Carrizo Plain,"MULTIPOLYGON (((-119.9167 35.24888, -119.91668..."
18265,Carrizo Plain,"MULTIPOLYGON (((-116.44282 33.63435, -116.4428..."
18381,Carrizo Plain,"MULTIPOLYGON (((-119.96042 35.34409, -119.9605..."


##### Plot them

In [40]:
### plot inyo
inyo_gdf.dissolve().hvplot(
    geo = True, tiles = 'EsriImagery',
    title = 'Inyo National Forest',
    fill_color = None, line_color = 'darkorange',
    frame_width = 600
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [41]:
### plot carrizo
carrizo_gdf.dissolve().hvplot(
    geo = True, tiles = 'EsriImagery',
    title = 'Carrizo National Monument',
    fill_color = None, line_color = 'darkorange',
    frame_width = 600
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [42]:
### combine
sites_gdf = gpd.GeoDataFrame(pd.concat([inyo_gdf, carrizo_gdf], ignore_index = True))
sites_gdf

,Loc_Nm,geometry
0,Inyo National Forest,"MULTIPOLYGON (((-118.55012 37.98427, -118.5469..."
1,Inyo National Forest,"MULTIPOLYGON (((-118.06059 36.60385, -118.0600..."
2,Inyo National Forest,"MULTIPOLYGON (((-119.02249 37.58457, -119.0221..."
3,Carrizo Plain,"MULTIPOLYGON (((-119.8591 35.1903, -119.85908 ..."
4,Carrizo Plain,"MULTIPOLYGON (((-119.85911 35.1903, -119.8591 ..."
5,Carrizo Plain,"MULTIPOLYGON (((-119.91665 35.24888, -119.9166..."
6,Carrizo Plain,"MULTIPOLYGON (((-119.60647 34.98708, -119.6064..."
7,Carrizo Plain,"MULTIPOLYGON (((-119.92409 35.26665, -119.9241..."
8,Carrizo Plain,"MULTIPOLYGON (((-119.98678 35.32143, -119.9867..."
9,Carrizo Plain,"MULTIPOLYGON (((-119.85911 35.1903, -119.8591 ..."
